# Building LLM applications: Notebook 03

# ReAct

## Initialize

In [1]:
import os
import dotenv
import json

from langchain_core.messages import HumanMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_core.tools import tool

from langchain.agents.output_parsers.react_single_input import ReActSingleInputOutputParser

from langchain_ollama import ChatOllama

In [2]:
MODEL = 'llama3.2'

In [ ]:
# Read fro `.env` file
dotenv.load_dotenv()

OLLAMA_URL = os.getenv('OLLAMA_URL')
print(f"Using Ollama server: {OLLAMA_URL if OLLAMA_URL else 'local'}")

## Step 1: Create tools (`str` parameters)

In [ ]:
@tool
def double_of(n):
    """Double of a number"""
    print(f"Double of (type: {type(n)}) {n}")
    n = int(n)
    print(f"Double of {n}")
    return 2 * n


@tool
def factorial_of(n):
    """Factorial of a number"""
    print(f"Factorial of (type: {type(n)}) {n}")
    n = int(n)
    fact = 1
    for i in range(1, n+1):
        fact *= i
    return fact


## Debug Runnable

In [5]:
class DebugRunnable(Runnable):
    def __init__(self, name):
        self.name = name

    """ A runnable that just prints the input and returns it """
    def invoke(self, input, config=None, **kwargs):
        print(f"DebugRunnable {self.name}: (type: {type(input)})")
        if isinstance(input, list):
            for i, item in enumerate(input):
                print(f"  {i}: {item}")
        elif isinstance(input, dict):
            print(json.dumps(input, indent=2, default=str))
        else:
            print(str(input))
        return input

## Excercise 01: Calling tools manually

In [ ]:
tools = [add, sub, double_of, factorial_of]
tools_map = {t.name.lower(): t for t in tools}
tools_map

In [ ]:
# Create the LLM object
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)

# Bind the tools to the LLM
llm_with_tools = llm.bind_tools(tools)

# Create a message
query = "What is the addition of 40 and 2?"
message = HumanMessage(query)
messages = [message]                # The list of messages to send to the LLM, now is only one message

ret = llm_with_tools.invoke(messages)
print(f"Type: {type(ret)}\n{ret}")

In [ ]:
# Take a look at the tool calls
ret.tool_calls

In [ ]:
# Call the tools, append the results to the messages list
for tool_call in ret.tool_calls:
    tool_name = tool_call['name'].lower()
    tool = tools_map.get(tool_name)
    if tool:
        tool_msg = tool.invoke(tool_call)
        print(tool_msg)
    print(f"Tool message: {tool_msg}")
    messages.append(tool_msg)

# Show the messages
messages

In [ ]:
# Invoke the LLM with the messages (including the tool results)
ret = llm_with_tools.invoke(messages)
print(ret.content)

## Excercise 02: AgentExecutor

In [ ]:
tools = [double_of, factorial_of]
tools_map = {t.name.lower(): t for t in tools}
tools_map

In [ ]:
from langchain.agents import create_tool_calling_agent, AgentExecutor


# Create the LLM object
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)

# Create a prompt object from the string template
prompt_template = """Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

prompt = PromptTemplate.from_template(prompt_template)
prompt

In [13]:
from langchain_core.agents import AgentAction

def format_log_to_str(
    intermediate_steps: list[tuple[AgentAction, str]],
    observation_prefix: str = "Observation: ",
    llm_prefix: str = "Thought: ",
) -> str:
    """Construct the scratchpad that lets the agent continue its thought process."""
    thoughts = ""
    for action, observation in intermediate_steps:
        thoughts += action.log
        thoughts += f"\n{observation_prefix}{observation}\n{llm_prefix}"
    return thoughts


In [14]:
agent = create_tool_calling_agent(llm, tools, prompt)

# agent_debug = DebugRunnable('Before agent') | agent | DebugRunnable('After agent')

llm_with_stop = llm.bind(stop=["\nObservation"])
agent = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
    )
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)


agent_executor = AgentExecutor(agent=agent, tools=tools)
# agent_executor = AgentExecutor(agent=agent_debug
#                                , tools=tools
#                                , return_intermediate_steps=True
#                                , handle_parsing_errors=False
#                                )

In [ ]:
query = "What is the factorial of 10?"
tool_names = ','.join([t.name for t in tools])

agent_executor.invoke({'input': query, 'tool_names': tool_names})